In [ ]:
# default_exp location_history_parse

# location_history_parse

I definitely jacked this from somewhere online but don't remember where now

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
from datetime import datetime
import pandas as pd
import numpy as np
import math

# old functions I jacked from somewhere online

In [ ]:
#export
def load_json_file(filename):
    with open(filename) as json_file:
        return json.load(json_file)

In [ ]:
#export
def parse_activities(activity_recs):
    flat_activities = {}
    for activity_rec in activity_recs:
        for activity in activity_rec['activity']:
            flat_activities[activity['type']] = activity['confidence']
    return flat_activities

In [ ]:
#export
def parse_json_file(filename, filters = {}):
    raw_json_data = load_json_file(filename)

    parsed_json_data = parse_json_data(raw_json_data)

    if filters != {}:
        filtered_json_data = filter_json_data(parsed_json_data, filters=filters)
    else:
        filtered_json_data = parsed_json_data

    sorted_json_data = sort_json_data(filtered_json_data, 'timestamp')

    return sorted_json_data

In [ ]:
#export
def parse_json_file_as_rows(filename, filters = {}):
    sorted_json_data = parse_json_file(filename, filters)
    rows = rowify_json_data(sorted_json_data)
    return rows

In [ ]:
#export
def parse_json_data(raw_json_data):
    parsed_data_points = []
    for data_point in raw_json_data["locations"]:
        parsed_data_points.append(parse_data_point(data_point))
    return {"locations": parsed_data_points}

In [ ]:
#export
def parse_data_point(data_point):
    data_point["timestamp"] = parse_timestamp(data_point.get("timestampMs"))
    data_point["lat"] = data_point.get("latitudeE7") / 1e7
    data_point["lon"] = data_point.get("longitudeE7") / 1e7
    data_point["accuracy"] = data_point.get("accuracy", np.nan)
    data_point["velocity"] = data_point.get("velocity", np.nan)
    data_point["heading"] = data_point.get("heading", np.nan)
    data_point["altitude"] = data_point.get("altitude", np.nan)
    data_point["vertical_accuracy"] = data_point.get("verticalAccuracy", np.nan)
#     data_point["activity"], data_point["activity_confidence"] = parse_activity(data_point.get("activity", ""))
    return data_point

In [ ]:
#export
def parse_activity(activity):
    if activity == "":
        return ["", ""]
    first_activity = activity[0]
    classifications = first_activity["activity"]
    best_classification = classifications[0]
    return [best_classification["type"], best_classification["confidence"]]

In [ ]:
#export
def filter_json_data(parsed_json_data, filters={}):
    filtered_data_points = []
    for data_point in parsed_json_data["locations"]:
        if check_against_filters(data_point, filters):
            filtered_data_points.append(data_point)
    return {"locations": filtered_data_points}

In [ ]:
#export
def sort_json_data(filtered_json_data, sort_key="timestamp"):
    filtered_json_data["locations"].sort(key=lambda data_point: data_point[sort_key])
    return filtered_json_data

In [ ]:
#export
def rowify_json_data(sorted_json_data):
    headers = [
        "timestamp",
        "latitude",
        "longitude",
        "accuracy",
        "velocity",
        "heading",
        "altitude",
        "vertical_accuracy",
        "activity",
        "activity_confidence",
    ]
    rows = [headers]
    for data_point in sorted_json_data["locations"]:
        row = [
            str(data_point.get("timestamp")),
            data_point["lat"],
            data_point["lon"],
            data_point["accuracy"],
            data_point["velocity"],
            data_point["heading"],
            data_point["altitude"],
            data_point["vertical_accuracy"],
            data_point["activity"],
            data_point["activity_confidence"],
        ]
        rows.append(row)
    return rows

In [ ]:
#export
def parse_timestamp(t):
    return datetime.utcfromtimestamp(int(t) / 1000)

In [ ]:
#export
def check_against_filters(data_point, filters):
    start = filters.get("start", False)
    end = filters.get("end", False)
    bbox = filters.get("bbox", False)

    # Skip data from before the provided start datetime
    if start and (data_point["timestamp"] <= start):
        return False

    # Skip data from after the provided end datetime
    if end and (end < data_point["timestamp"]):
        return False

    # Skip data_points outside of bounding box
    if (bbox
        and ((data_point["lat"] < bbox["min_lat"])
            or (data_point["lat"] > bbox["max_lat"])
            or (data_point["lon"] < bbox["min_lon"])
            or (data_point["lon"] > bbox["max_lon"]))):
        return False

    # Skip data that hasn't been assigned an activity category
    if len(data_point["activity"]) == 0:
        return False

    # If data_point passes all filters, return True
    return True

In [ ]:
#export
def d(timestamp):
    return str(parse_timestamp(timestamp))

In [ ]:
#export
def create_timestamp(datetime):
    return datetime.strftime("%s%f")

In [ ]:
#export
def t(datetime):
    return create_timestamp(datetime)

# new functions for parsing to pandas df

In [ ]:
#export
def parse_google_location_history_json(filepath):
    raw = load_json_file(filepath)
    raw = raw['locations']
    parsed_data_points = []
    for data_point in raw:
        parsed_data_point = parse_data_point(data_point)
        if 'activity' in data_point.keys():
            parsed_activities = parse_activities(data_point['activity'])
            parsed_data_point.update(parsed_activities)
            parsed_data_point.pop('activity')
        parsed_data_points.append(parsed_data_point)
    return parsed_data_points

In [ ]:
#export
def parsed_loc_hist_to_df(parsed_data_points):
    df = pd.DataFrame().from_records(parsed_data_points)
    
    #calculate odds of being moving
    df['MOVING'] = 100 - df.loc[:,['STILL','UNKNOWN']].sum(axis=1)
    df.loc[df['STILL'].isna() & df['UNKNOWN'].isna(), 'MOVING'] = np.nan
    
    #add delta position
    df.loc[1:,'dlat'] = df['lat'].diff()
    df.loc[1:,'dlon'] = df['lon'].diff()
    df['dpos'] = (df['dlat']**2 + df['dlon']**2) ** 0.5
    
    df.columns = [col.lower() for col in df.columns]
    df = df.rename(columns={'timestamp':'t'})
    
    front_cols = [
        't',
        'lat',
        'lon',
        'dpos',
        'velocity',
        'heading',
        'moving',
        'still',
        'unknown',
        'in_vehicle',
        'on_bicycle',
        'on_foot',
        'walking',
        'running',
        'dlat',
        'dlon',
    ]
    cols = df.columns.to_list()
    for col in reversed(front_cols):
        cols.remove(col)
        cols.insert(0,col)
    df = df.loc[:, cols]
    
    df = df.set_index('t')
#     df.index = df.index.tz_localize('UTC', ambiguous='infer').tz_convert('US/Pacific')
    df.index = df.index - pd.Timedelta('8H')
    return df

In [ ]:
#export
def google_location_history_to_df(filepath):
    return parsed_loc_hist_to_df(parse_google_location_history_json(filepath))

# for adding new info to parsed df

In [ ]:
#export
def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [ ]:
#export
def coords_to_m_from_hpsc(coord):
    hpsc_coord = (37.856668, -122.257166)
    return haversine(coord, hpsc_coord)

In [ ]:
#export
def get_dt(df):
    dt = df.reset_index()['t'].astype(int).diff() * 1e-9
    return dt.values

In [ ]:
#export
def add_dt_and_velocity(df):
    '''warning takes hella long
    '''
    df.loc[:,'dt'] =  get_dt(df)
    df.index = df.index.tz_localize(None)

    m_from_hpsc = []
    for lat, lon in zip(df['lat'].values, df['lon'].values):
        coord = (lat, lon)
        d = coords_to_m_from_hpsc(coord)
        m_from_hpsc.append(d)

    df['m_from_hpsc'] = m_from_hpsc
    df['km_from_hpsc'] = df['m_from_hpsc'] * 1e-3

    df['dpos_hpsc'] = df['m_from_hpsc'].diff()

    latlon = df.loc[:,['lat','lon']]
    latlon_lag = latlon.shift(1)

    dpos_ms = []
    for i in tqdm(range(len(latlon))):
        coord = (latlon.iloc[i]['lat'], latlon.iloc[i]['lon'])
        prev_coord = (latlon_lag.iloc[i]['lat'], latlon_lag.iloc[i]['lon'])
        dpos_ms.append(haversine(coord, prev_coord))
        
    df['dposm'] = dpos_ms
    df['mpers'] = df['dposm'] / df['dt']
    return df

In [ ]:
#export
def get_is_trip_bool(
        data, 
        min_vel=0, 
        max_vel=35,
        max_dt=120,
        max_dposm=300,
    ):
    moving = (min_vel <= data['mpers'])
    not_too_fast = (data['mpers'] <= max_vel)
    smooth_in_time = (data['dt'] <= max_dt)
    smooth_in_space = (data['dposm'] <= max_dposm)
    is_trip = moving & not_too_fast & smooth_in_time & smooth_in_space
    return is_trip

In [ ]:
#export
def get_trip_time_bools(data, is_trip):
    ind_diff = is_trip.astype(int).diff().fillna(0)
    trip_starts = (ind_diff == 1)
    trip_ends = (ind_diff == -1)
    trips = (ind_diff.apply(lambda x: max(x, 0)).cumsum() * is_trip).astype(int)
    return trip_starts, trip_ends, trips

In [ ]:
#export
def split_trips_into_lines(data):
    # split into trips
    trip_starts_ind = np.where(data['trip_starts'])[0]
    trips = np.split(data.loc[:,['lon','lat',]].values, trip_starts_ind)
    return trips

In [ ]:
#export
def get_big_bay_lims():
    lims = {}
    lims['N'] = 38
    lims['S'] = 37.6
    lims['E'] = -122.15
    lims['W'] = -122.6
    return lims

In [ ]:
#export
def get_small_bay_lims():
    lims = {}
    lims['N'] = 37.9
    lims['S'] = 37.75
    lims['E'] = -122.2
    lims['W'] = -122.34
    return lims

In [ ]:
#export
def spatial_filter_df(df, lims):
    ind = (df['lat'] < lims['N']) & (lims['S'] < df['lat']) 
    ind = ind & (lims['W'] < df['lon']) & (df['lon'] < lims['E'])
    return ind

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_gram.ipynb.
Converted 01_gram_scratch.ipynb.
Converted 02_write.ipynb.
Converted 03_quiver.ipynb.
Converted 04_graph_grams.ipynb.
Converted 05_blender_prep.ipynb.
This cell doesn't have an export destination and was ignored:
h
Converted 06_turtle_scratch.ipynb.
Converted 07_location_history_parse.ipynb.
Converted 08_location_hist_scratch.ipynb.
Converted 09_location_hist_plots.ipynb.
Converted index.ipynb.
